In [ ]:
#!pip install simple_salesforce

In [5]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [6]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [7]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [8]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Prepare ETL for the Course data

In [9]:
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(30)

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0
5,6,CIS-438,Database Administration,5,0
6,7,CIS-430,Business Systems Programming I,5,0
7,8,CIS-435,Business Systems Programming II,5,0


In [10]:
course_data_df.rename(columns={
    'ID_Course':'ID_Course__c',
    'CourseCode':'Course_Code__c',
    'CourseName':'Course_Name__c',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

,ID_Course__c,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [ ]:
#course_data_df = course_data_df[['Course_Code__c', 'Course_Name__c', 'Credit_Hours__c', 'Boot_Camp_Course__c']]
#course_data_df.head()

In [11]:
course_data_records = course_data_df.to_dict('records')
course_data_records

[{'ID_Course__c': 1,
  'Course_Code__c': 'BC-DATAVIZ',
  'Course_Name__c': 'Data Visualization and Analytics',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'ID_Course__c': 2,
  'Course_Code__c': 'BC-WEBDEV',
  'Course_Name__c': 'Full Stack Web Development',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'ID_Course__c': 3,
  'Course_Code__c': 'BC-UIUX',
  'Course_Name__c': 'User Interface/User Experience',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'ID_Course__c': 4,
  'Course_Code__c': 'CIS-349',
  'Course_Name__c': 'Introduction to Databases',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'ID_Course__c': 5,
  'Course_Code__c': 'CIS-405',
  'Course_Name__c': 'Database Programming',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'ID_Course__c': 6,
  'Course_Code__c': 'CIS-438',
  'Course_Name__c': 'Database Administration',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'ID_Course__c': 7,
  'Course_Code__c': 'CIS-430',
  'Course_

In [12]:
for rec in course_data_records:

    record = {
        'ID_Course__c': rec['ID_Course__c'],
        'Course_Code__c': rec['Course_Code__c'],
        'Course_Name__c': rec['Course_Name__c'],
        'Credit_Hours__c': rec['Credit_Hours__c'],
        'Boot_Camp_Course__c': rec['Boot_Camp_Course__c'],
    }
    
    try:
        sf.Course__C.create(record)
    except Exception as e:
        print(e)
        

## Create Course Lookup Table
You will use this later to crosswalk the course code with the primary key from the `Course` table

It is important to note that we will be querying **Salesforce** to retrieve the record IDs 

In [26]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT ID_Course__c, Name FROM Course__c")
for row in data:
    rec = {
        'Course_Record_ID__c': row['Name'], # this is a critical line of code
        'ID_Course__c': row['ID_Course__c']
    }
    course_lookup_list.append(rec)
    
course_lookup_list

[{'Course_Record_ID__c': 'a043h00000Lm91D', 'ID_Course__c': 6.0},
 {'Course_Record_ID__c': 'a043h00000Lm913', 'ID_Course__c': 4.0},
 {'Course_Record_ID__c': 'a043h00000Lm91N', 'ID_Course__c': 8.0},
 {'Course_Record_ID__c': 'a043h00000Lm90t', 'ID_Course__c': 2.0},
 {'Course_Record_ID__c': 'a043h00000Lm918', 'ID_Course__c': 5.0},
 {'Course_Record_ID__c': 'a043h00000Lm90o', 'ID_Course__c': 1.0},
 {'Course_Record_ID__c': 'a043h00000Lm91I', 'ID_Course__c': 7.0},
 {'Course_Record_ID__c': 'a043h00000Lm90y', 'ID_Course__c': 3.0}]

In [27]:
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

,Course_Record_ID__c,ID_Course__c
0,a043h00000Lm91D,6.0
1,a043h00000Lm913,4.0
2,a043h00000Lm91N,8.0
3,a043h00000Lm90t,2.0
4,a043h00000Lm918,5.0
5,a043h00000Lm90o,1.0
6,a043h00000Lm91I,7.0
7,a043h00000Lm90y,3.0


In [28]:
# Query the `Class` table from MySQL

class_data_df = pd.read_sql('SELECT * FROM class', conn)
class_data_df.head()

,ID_Class,ID_Course,Section,StartDate,EndDate
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [30]:
class_data_df.rename(columns={
    'ID_Class':'ID_Class__c',
    'ID_Course':'ID_Course__c',
    'Section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    'CourseCode':'Course_Code__c'
}, inplace=True)

class_data_df

,ID_Class__c,ID_Course__c,Section__c,Start_Date__c,End_Date__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


## Join the Class DataFrame with the Course lookup table
This join is necessary to successfully lookup the foreign key for the Course table 

In [34]:
class_data_df = pd.merge(class_data_df, course_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

class_data_df.head()

,ID_Class__c,ID_Course__c,Section__c,Start_Date__c,End_Date__c,Course_Record_ID__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14,a043h00000Lm90o
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08,a043h00000Lm90o
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07,a043h00000Lm90t
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07,a043h00000Lm90y


In [35]:
class_data_df
class_data_df['Start_Date__c'] = pd.to_datetime(class_data_df['Start_Date__c']).dt.date
class_data_df['End_Date__c'] = pd.to_datetime(class_data_df['End_Date__c']).dt.date

class_data_df.head()


,ID_Class__c,ID_Course__c,Section__c,Start_Date__c,End_Date__c,Course_Record_ID__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14,a043h00000Lm90o
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08,a043h00000Lm90o
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07,a043h00000Lm90t
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07,a043h00000Lm90y


In [36]:
class_data_records = class_data_df.to_dict(orient='records')
class_data_records

[{'ID_Class__c': 1,
  'ID_Course__c': 1,
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': datetime.date(2020, 3, 16),
  'End_Date__c': datetime.date(2020, 3, 14),
  'Course_Record_ID__c': 'a043h00000Lm90o'},
 {'ID_Class__c': 2,
  'ID_Course__c': 1,
  'Section__c': 'GWDC201805DATA3',
  'Start_Date__c': datetime.date(2018, 5, 15),
  'End_Date__c': datetime.date(2018, 11, 8),
  'Course_Record_ID__c': 'a043h00000Lm90o'},
 {'ID_Class__c': 3,
  'ID_Course__c': 2,
  'Section__c': 'GWARL201905WEB3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7),
  'Course_Record_ID__c': 'a043h00000Lm90t'},
 {'ID_Class__c': 4,
  'ID_Course__c': 3,
  'Section__c': 'GWARL201905UIUX3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7),
  'Course_Record_ID__c': 'a043h00000Lm90y'}]

## Insert `Class` Records into Salesforce

In [37]:
for rec in class_data_records:
 
    record = {
        'ID_Class__c': rec['ID_Class__c'],
        'ID_Course__c': rec['ID_Course__c'],
        'Course_Record_ID__c': rec['Course_Record_ID__c'],
        'Section__c': rec['Section__c'],
        'Start_Date__c': str(rec['Start_Date__c']),
        'End_Date__c': str(rec['End_Date__c']),
    }
    
    try:
        sf.Class__C.create(record)
    except Exception as e:
        print(e)

## Example of Deleting Records

Select the IDs of the records first and then process the results.

Ultimately, you want a list of IDs in the end.


In [ ]:
class_records = sf.query("SELECT Id FROM Course__c")
recs_to_delete = [{'Id': r['Id']} for r in class_records['records']]
recs_to_delete

In [ ]:
sf.bulk.Course__c.delete(recs_to_delete)